In [85]:
import mysql.connector
import json
import uuid
import pandas as pd

In [86]:
# Connection parameters
host = 'localhost'
user = 'root'
password = 'Ncgncg1102@'
database = 'stock_prediction'

In [87]:
data="C:\\Users\\GIANG\DAC_Project\\Project_python\\2024-dn-python-stockprediction\\crawlData\\data.json"
with open(data,'r') as file:
  data=json.load(file)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\GIANG\AppData\Local\Temp\ipykernel_704\2085821978.py:1: SyntaxWarning: invalid escape sequence '\D'
  data="C:\\Users\\GIANG\DAC_Project\\Project_python\\2024-dn-python-stockprediction\\crawlData\\data.json"


In [88]:
symbols=list(data.keys())
name_symbols=["Ngân hàng TMCP Á Châu","Ngân hàng TMCP Đầu tư và Phát triển Việt Nam","Ngân hàng TMCP Công Thương Việt Nam","Ngân hàng TMCP Xuất nhập khẩu Việt Nam","Ngân hàng TMCP Phát triển TP.HCM","Ngân hàng TMCP Quân Đội","Ngân hàng TMCP Hàng hải Việt Nam","Ngân hàng TMCP Phương Đông","Ngân hàng TMCP Sài Gòn – Hà Nội","Ngân hàng TMCP Đông Nam Á","Ngân hàng TMCP Sài Gòn Thương Tín","Ngân hàng TMCP Kỹ thương Việt Nam","Ngân hàng TMCP Tiên Phong","Ngân hàng TMCP Ngoại Thương Việt Nam","Ngân hàng TMCP Quốc tế Việt Nam","Ngân hàng TMCP Việt Nam Thịnh Vượng","Ngân hàng TMCP Quốc Dân","Ngân hàng TMCP Bắc Á"]

In [89]:
stocks=[]
for symbol in symbols:
  stocks.append(list(data[symbol].values()))
previous_close_prices=[]
for stock in stocks:
      previous_close_prices.append(eval(stock[0])["close"])


In [90]:
stock_ids=[]
for i in range (len(stocks)):
    stock_ids.append(str(uuid.uuid4()))

In [91]:
#create data insert to stocklist
data_stock_list_inserts=[]
for i in range(len(stocks)):
    data_stock_list_inserts.append((stock_ids[i],symbols[i],name_symbols[i],float(previous_close_prices[i].replace(',', ''))))

In [92]:
#convert dict to dataframe
from ast import literal_eval  
def convert_str_to_float(str_num):
    return float(str_num.replace(',', ''))
def convert_dict_to_df(data_dict):
    data = {key: literal_eval(value) for key, value in data_dict.items()}
    df = pd.DataFrame.from_dict(data, orient='index')
    df.index = pd.to_datetime(df.index, format='%d/%m/%Y')
    df = df.map(convert_str_to_float)
    df=df.sort_index(ascending=True)
    return df

In [93]:
data_historys=[]
for symbol in symbols:
  data_historys.append(data[symbol])
data_history_dfs=[]
for data_history in data_historys:
    data_history_dfs.append(convert_dict_to_df(data_history))

In [94]:
# Create a list to store the data inserts
data_history_inserts = []

for stockid, dataframe in zip(stock_ids, data_history_dfs):
    for index, row in dataframe.iterrows():
        
        date = row.name.date()  
        open_price = row['open']  
        high_price = row['high'] 
        low_price = row['low']  
        close_price = row['close']  
        volume = row['volume']  
        record = (stockid, str(date), open_price, high_price, low_price, close_price, volume)
       
        data_history_inserts.append(record)

In [95]:
# #insert to db
# try:
#     conn = mysql.connector.connect(
#         host=host,
#         user=user,
#         password=password,
#         database=database
#     )

#     if conn.is_connected():
#         print("Connected to MySQL")
        
#         cursor = conn.cursor()
#         insert_stocklist_query = "INSERT INTO stocklist (stockid, symbol, company_name,previous_close_price) VALUES (%s, %s, %s,%s)"
#         cursor.executemany(insert_stocklist_query, data_stock_list_inserts)

#         insert_stock_hist_query = "INSERT INTO stockhistory (stockid, date, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
#         cursor.executemany(insert_stock_hist_query, data_history_inserts)
#         conn.commit()

#         cursor.close()
#         conn.close()

# except mysql.connector.Error as e:
#     print(f"Error: {e}")
# finally:
#     if conn.is_connected():
#         cursor.close()
#         conn.close()
#         print("MySQL connection closed")